[toc]

# DL-学习笔记-1-各种优化器的比较

## 标准的 Gradient Descent

$$
\theta \leftarrow \theta - \eta \frac{1}{N}\sum_{i=1}^{N} \nabla L(y_i, \hat{y}_i; \theta)
$$ 

需要将所有的样本处的梯度都累加起来，计算速度会比较慢

### SGD

不是使用全部的样本，而是只随机抽取一个样本来更新梯度。可以减少计算量，加快更新速度。

$$
\theta \leftarrow \theta - \eta \nabla L(y_i, \hat{y}_i; \theta)
$$ 

### Mini-Batch Gradient Descent

是上面两个方法的折中。取一个Batch来更新梯度，然后只累加 Batch 中的样本梯度。

可以看到，标准的Gradient Descent 和 SGD 是 Mini-Batch Gradient Descent 的一个特例。
当 Mini-Batch 只取一个样本时，就是 SGD；当 Mini-Batch 取所有样本时，就是标准的 Gradient Descent

### 问题

1. 震荡问题：根据 batch 来进行更新，由于 batch 之间会有一些差异，因此可以导致更新不稳定，出现来回震荡的情况，这会降低收敛的速度，甚至导致不收敛。
1. 稀疏特征的学习率问题：所有的参数都使用了相同的学习率，但是，对于那些比较稀疏的特征来说，更新次数是比较少的，对于这些特征相关的参数，我们希望它们的学习率能大一些。

---

**提示**：为了公式简单，之后的优化算法中，我们使用 $g(\theta)$  来代替梯度。

$g(\theta)$  的不同取值代表了不同的版本。如
- $g(\theta) = \frac{1}{N} \sum_{i=1}^{N} \nabla L(y_i, \hat{y}_i; \theta)$ 表示的是标准Gradient Descent版本；
- $g(\theta) = \nabla L(y_i, \hat{y}_i; \theta)$  表示 SGD版本等

## 动量优化

Gradient Descent 优化的一个方向是动量优化。有 Momentum 和 Nestrove Accelarated Gradient

### Momentum

在更新时不仅仅考虑本次更新方向，还考虑之前的更新方向。

$$
\begin{aligned}
& v \leftarrow  \alpha v + \eta g(\theta)  \\
& \theta \leftarrow \theta - v
\end{aligned}
$$ 

其中是$v$ 表示之前的速度，初始化为 0。

#### Momentum 的作用

Momentum 主要有下面几个作用

1. 更新参数时，不但考虑了本次更新，还考虑了上一次的更新，因此可以减少震荡，加速收敛。借 [ 2 ] 中的一张图

![](https://gitee.com/EdwardElric_1683260718/picture_bed/raw/master/img/20200716171023.png)

可以看到，使用了 momentum 的更新路径更加平滑，速度也比普通的 SGD 要快一些。

2. 有利于函数在平原处也可以进行更新，有助于跳过 local minima，借李宏毅老师的一张图

![](https://gitee.com/EdwardElric_1683260718/picture_bed/raw/master/img/20200716171609.png)

#### Pytorch 中的 momentum

注意，上面的公式是原始的 momentum 的公式。在许多深度学习的框架里，对上面的公式有所修改，变成了：

$$
v \leftarrow \alpha  v  + g(\theta) \\
\theta \leftarrow \theta - \eta v
$$ 

![](https://gitee.com/EdwardElric_1683260718/picture_bed/raw/master/img/20200703184811.png)

不知道这样做有什么好处。可能是这种形式可以直接从 sgd 修改，从而减少代码量。不知道这两种形式是否等价。

### NAG

更新速度的时间不仅考虑过去，还考虑未来

$$
\begin{aligned}
    & v \leftarrow - \eta g(\theta + \alpha v)  + \alpha v \\
    & \theta \leftarrow  \theta + v
\end{aligned}
$$ 

可以看到根据未来进行的一个校正。

## 自适应学习

对Gradient Descent进行优化的另一个方向是调整学习率的角度。主要的方法有 Adagrad、RMSprop

### Adagrad

Adagrad 是思路是对学习率进行一个加权。权重是累计的梯度平方和

$$
\begin{aligned}
    & h \leftarrow h + g^2 \\
    & \theta \leftarrow \theta - \eta \frac{1}{\sqrt{h + \epsilon} }g
\end{aligned}
$$ 

#### 作用

Adagrad 可以解决稀疏特征的问题。其思想是对那么更新多的参数的学习率进行惩罚，使其更新慢一些。从而让更新快的、更新频繁的参数等一等更新慢的、更新不频繁的参数（而稀疏特征对应的参数更新就不频繁）

#### 缺点

历史信息不断累加，导致学习率越来越小，不足以维持学习到一个最优的参数，可能导致模型欠拟合。

### RMSprop

RMSprop 对 Adagrad 进行了一个小小的改变。将梯度平方和的累加变成了梯度平方和的滑动平均。

Adagrad 会一直记得之前的信息，可能会导致到后面学习率太小，导致学习率不更新，使用模型欠拟合。

而 RMSprop 会逐渐忘记之前的信息，因此不会有这个问题。

$$
\begin{aligned}
    & h \leftarrow \alpha h + (1 - \alpha)g^2 \\
    & \theta \leftarrow \theta - \eta \frac{1}{\sqrt{h + \epsilon} }g
\end{aligned}
$$ 

![花书 rmsprop](https://gitee.com/EdwardElric_1683260718/picture_bed/raw/master/img/20200428132944.png)

## 集大成者 Adam

Adam 相当于将上面和两个方向的优化结合到了一起，并做出了一个修改。既考虑了动量的方向，又调整了学习率。


$$
\begin{aligned}
    & s \leftarrow \alpha s + (1-\alpha ) g \\
    & r \leftarrow \beta r + (1- \beta) g^2 \\
    & \hat{s} \leftarrow  \frac{s}{ 1 - \alpha^t} \\
    & \hat{r} \leftarrow  \frac{r}{ 1 - \beta^t} \\
    & \theta \leftarrow \theta - \eta \frac{1}{\sqrt{r + \epsilon} }  s
\end{aligned}
$$ 
 ![花书 Adam](https://gitee.com/EdwardElric_1683260718/picture_bed/raw/master/img/20200428134119.png)

# References

1. 花书
2. [An overview of gradient descent optimization algorithms](https://ruder.io/optimizing-gradient-descent/index.html)
3. [机器学习：各种优化器Optimizer的总结与比较_人工智能_SanFancsgo的博客-CSDN博客](https://blog.csdn.net/weixin_40170902/article/details/80092628)